In [24]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from collections import defaultdict
import re
import json
import os
import time
import arrow
import random

import numpy as np

import subprocess
import zipfile

from attraction import Attraction
from review import Review
from user import User

In [25]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
options.add_argument('--incognito')
options.add_argument('--start-maximized')
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)

# if headless:
#     options.add_argument('--headless')

In [26]:
driver = webdriver.Chrome('webdriver/chromedriver', options=options)

driver.get('https://www.tripadvisor.com.au/Attraction_Review-g255060-d257278-Reviews-Sydney_Opera_House-Sydney_New_South_Wales.html')

In [29]:
traveller_ratings = ['Excellent', 'Very good', 'Average', 'Poor', 'Terrible']
traveller_types = ['Families', 'Couples', 'Solo', 'Business', 'Friends']
times_of_year=['Mar-May', 'Jun-Aug', 'Sep-Nov', 'Dec-Feb']


def tick_checkbox_item(traveller_rating=None, traveller_type='Solo', time_of_year=None,  language='English', max_attempts=3):
    
    traveller_rating_dataname = 'ta_rating'
    traveller_type_dataname = 'traveler_filter'
    time_of_year_dataname = 'season'
    language_dataname = 'language'
    
    d = {'traveller_rating': {'data-name': 'ta_rating',
                              'input-values': {'Excellent': '5',
                                               'Very good': '4',
                                               'Average': '3',
                                               'Poor': '2',
                                               'Terrible': '1'}},
        'traveller_type': {'data-name': 'traveler_filter',
                           'input-values': {'Families': '3',
                                            'Couples': '2',
                                            'Solo': '5',
                                            'Business': '1',
                                            'Friends': '4'}},
        'time_of_year': {'data-name': 'season',
                         'input-values': {'Mar-May': '1',
                                          'Jun-Aug': '2',
                                          'Sep-Nov': '3',
                                          'Dec-Feb': '4'}},
        'language': {'data-name': 'language',
                     'input-values': {'English': 'en',
                                      'Japanese': 'ja'}}}
    
    times_tried = 0
    
    traveller_rating_selected = False
    traveller_type_selected = False
    time_of_year_selected = False
    language_selected = False
    
    while (times_tried <= max_attempts) and (not traveller_rating_selected):
        
        times_tried += 1
        
        # filter by traveller rating
        
        if traveller_rating:
            
            tr_pick = d['traveller_rating']['input-values'][traveller_rating]
            print('tr_pick=',tr_pick)
            dname = d["traveller_rating"]["data-name"]
            
            print('dataname=', d["traveller_rating"]["data-name"])
            st = f'div.choices[data-name="{dname}"]>div[data-value="{tr_pick}"]'
            print(st)
            try:
                e = WebDriverWait(driver, 10) \
                    .until(EC.element_to_be_clickable((By.CSS_SELECTOR, st)))
                print('got element! clicking..')
                e.click()
            except:
                print(f'failed to click on {traveller_rating} filter..')
            
        
#         try:
#             WebDriverWait(driver, 10) \
#                 .until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
#                                                f'div[data-tracker="{item_name}"]'))).click()
#         except:
            
#             print(f'failed to click on {item_name} filter..')
            st2 = st + '>input[checked="checked"]'
            print(st2)
        
            try:
                WebDriverWait(driver, 10) \
                    .until(EC.presence_of_element_located((By.CSS_SELECTOR, st2)))
                traveller_rating_selected = True
            
            except:
            
                print(f'failed to select {traveller_rating}, re-trying...')
            
#     if is_selected: 
        
#         nrevs = 0
        
#         try:
            
#             WebDriverWait(driver, 10) \
#                 .until(EC.presence_of_element_located((By.CSS_SELECTOR, 
#                                                    'input#filters_detail_language_filterLang_en[checked="checked"]')))
            
#             nrevs = int(re.sub(r'[(,)]', '', driver.find_element_by_css_selector('label[for="filters_detail_language_filterLang_en"]>span.count').text))
            
#             print(f'available reviews by {item_name} travellers: {nrevs:,}')
            
#         except:
            
#             langs = [_ for _ in driver.find_element_by_css_selector('div[data-prwidget-name="filters_detail_language"]').text.split('\n') if '(' in _]
            
#             print(f'no English {item_name} reviews?')
            
#             if langs:
#                 print(f'available review languages: {" | ".join(langs)}')
#             else:
#                 print('there are no reviews in any language')          

In [30]:
tick_checkbox_item(traveller_rating='Average')

tr_pick= 3
dataname= ta_rating
div.choices[data-name="ta_rating"]>div[data-value="3"]
got element! clicking..
div.choices[data-name="ta_rating"]>div[data-value="3"]>input[checked="checked"]
